# Apartment price prediction with sklearn

### Libraries and settings

In [ ]:
# Libraries
#  %pip install numpy pandas matplotlib scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

### Import the apartment data

In [ ]:
# Read and select variables
df_orig = pd.read_csv("original_apartment_data_analytics_hs24.csv", sep=",", encoding='utf-8')

# Remove missing values
df = df_orig.dropna()
# Remove duplicates
df = df.drop_duplicates()

print(df.shape)
df.head(10)

In [ ]:
# Meaning of variables:
# bfs_number: official municipality id
# bfs_name: official municipality name
# pop: number of residents (=population)
# pop_dens: population density (pop per km2)
# frg_pct: percentage foreigners
# emp: numer of employees

df.columns

### Train/Test splitting

In [19]:
# Create train and test samples
X_train, X_test, y_train, y_test = train_test_split(df[['rooms', 'area', 'pop', 'pop_dens', 'frg_pct',
                                                        'emp', 'tax_income']], 
                                                         df['price'], 
                                                        test_size=0.20, 
                                                        random_state=42)

<hr >

### Training the models

#### Questions
- What does the score represent? 
- How is the root mean squared error calculated?
- How good are the model performing? 1=bad, 10=perfect
- Is the LinearRegression model overfitting or underfitting?
- Is the RandomForestRegressor overfitting or underfitting?


In [ ]:
# train linear_model = LinearRegression()
linear_model = LinearRegression()

# Fit the model
linear_model.fit(X_train, y_train)
print("Train score: ", linear_model.score(X_train, y_train))
print("Test score: ", linear_model.score(X_test, y_test))

print("Train RMSE: ", root_mean_squared_error(y_train, linear_model.predict(X_train)))
print("Test RMSE: ", root_mean_squared_error(y_test, linear_model.predict(X_test)))
 

Kurz-Zusammenfassung:

- Score (R²): 56,5 % (Train) und 42,9 % (Test) – Das Modell erklärt die Daten nur mässig gut.
- RMSE: 831,65 (Train) und 974,24 (Test) – Hohe Fehlerquote, besonders bei Testdaten.
- Modellbewertung: Mittelmässige Leistung, eher Underfitting, da das Modell nicht komplex genug ist, um Muster zu erfassen.
- Performance: Modell auf etwa 4 einstufen.
- Empfehlung: Bessere Feature-Auswahl, Hyperparameter-Tuning oder ein komplexeres Modell wie RandomForest verwenden.

In [ ]:
# train random_forest_model = RandomForestRegressor()
random_forest_model = RandomForestRegressor(random_state=42)

# Fit the model
random_forest_model.fit(X_train, y_train)
print("Train score: ", random_forest_model.score(X_train, y_train))
print("Test score: ", random_forest_model.score(X_test, y_test))

print("Train RMSE: ", root_mean_squared_error(y_train, random_forest_model.predict(X_train)))
print("Test RMSE: ", root_mean_squared_error(y_test, random_forest_model.predict(X_test)))

Kurz-Zusammenfassung:

- Score (R²): 83,2 % (Train) und 78,6 % (Test) – Gute Erklärkraft des Modells.
- RMSE: 512,34 (Train) und 589,71 (Test) – Deutlich geringere Abweichungen als zuvor.
- Modellbewertung: Gute Leistung, Modell ist gut angepasst, kein Overfitting erkennbar.
- Performance: Ca. 8.
- Empfehlung: Modell ist solide. Kleinere Optimierungen oder Feature-Erweiterungen könnten die Performance weiter steigern.

<hr >

### Random Forest feature importance

In [ ]:
cols = random_forest_model.feature_names_in_

# Derive feature importance from random forest
importances = random_forest_model.feature_importances_
std         = np.std([tree.feature_importances_ for tree in random_forest_model.estimators_], axis=0)
indices     = np.argsort(importances)[::-1]

# Print col-names and importances-values
print( cols[indices] )
print( importances[indices] )

# Barplot with feature importance
df_fi = pd.DataFrame({'features':cols,'importances': importances})
df_fi.sort_values('importances', inplace=True)
df_fi.plot(kind='barh', 
           y='importances',
           x='features', 
           color='darkred', 
           figsize=(6,3))

Feature-Bedeutung:

- Das wichtigste Feature für das Modell ist "area" mit einer Gewichtung von 62,25 %. Das bedeutet, dass die Wohnfläche (area) den grössten Einfluss auf die Vorhersage hat.
- Das zweitwichtigste Feature ist "rooms" mit 11,20 %, gefolgt von "pop_dens" (Bevölkerungsdichte) mit 8,10 % und "emp" (Beschäftigungsquote) mit 8,05 %.
- "pop", "tax_income" und "frg_pct" haben einen deutlich geringeren Einfluss, jeweils unter 7 %.

Interpretation:

- Die Wohnfläche ist der dominierende Faktor für die Vorhersage, was darauf hindeutet, dass grössere Wohnungen tendenziell teurer sind oder stärker mit der Zielvariable korrelieren.
- Die Anzahl der Zimmer und die Bevölkerungsdichte spielen eine moderate Rolle, während dem Einkommen und dem Ausländeranteil (frg_pct) eine geringere Bedeutung zukommt.

### Calculate the residuals

In [23]:
# make predictions
y_train_predict = random_forest_model.predict(X_train)
residuals = y_train - y_train_predict

In [ ]:
# Calculate residuals

plt.figure(figsize=(10, 6))
plt.scatter(y_train, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Actual Prices')
plt.ylabel('Residuals')
plt.title('Residuals vs Actual Prices')
plt.show()

Was zeigt der Plot?

- X-Achse: Tatsächliche Preise (Actual Prices)
- Y-Achse: Residuen (Differenz zwischen den vorhergesagten und den tatsächlichen Werten)
- Jeder Punkt repräsentiert eine Vorhersage des Modells im Vergleich zum tatsächlichen Wert. Die Residuen sind der Fehler der Vorhersage:

- Positiver Residualwert → Vorhersage war zu niedrig (tatsächlicher Preis war höher)
- Negativer Residualwert → Vorhersage war zu hoch (tatsächlicher Preis war niedriger)
- Die rote gestrichelte Linie zeigt den Nullpunkt, an dem die Vorhersagen perfekt wären.

Interpretation der Verteilung:

Streuung:

Die Residuen sind bei niedrigeren Preisen relativ gleichmässig um die Null-Linie verteilt, was auf eine gute Modellanpassung in diesem Bereich hinweist.
Bei höheren Preisen nimmt die Streuung jedoch zu, was bedeutet, dass das Modell bei teureren Wohnungen weniger präzise ist.

Kein klares Muster:

Es gibt keine eindeutige Krümmung oder systematische Abweichung. Das deutet darauf hin, dass keine starke Nicht-Linearität vorliegt.
Falls ein "U"- oder "umgekehrtes U"-Muster sichtbar wäre, würde das auf eine systematische Unter- oder Überschätzung hinweisen.

Ausreisser:

Einzelne Punkte mit extrem hohen Residuen (besonders über +3000 und unter -2000) sind Ausreißer, die möglicherweise durch ungewöhnliche Eigenschaften der Wohnungen oder Fehler in den Daten verursacht wurden.

Mögliche Massnahmen zur Verbesserung:

- Feature-Engineering: Zusätzliche Merkmale wie die Nähe zu wichtigen Einrichtungen könnten die Vorhersagen verbessern.
- Transformation: Bei teureren Objekten könnte eine Log-Transformation der Preise helfen, die Skala zu glätten.
- Modell-Optimierung: Ein komplexeres Modell wie Gradient Boosting oder XGBoost könnte die Genauigkeit verbessern.

In [ ]:
# Plot histogram of residuals
fig = plt.figure( figsize=(8,4))
n, bins, patches = plt.hist(x=residuals, 
                            bins=20, 
                            color='blue',
                            alpha=0.5
                   )

# Set labels
plt.xlabel('residuals', fontsize=10, labelpad=10)
plt.ylabel('frequency', fontsize=10, labelpad=10)
plt.title('Histogram of model residuals', fontsize=12, pad=10)

plt.show()

Das Histogramm zeigt eine ungefähr symmetrische Verteilung der Residuen um null, was auf ein gut angepasstes Modell hinweist. Die meisten Fehler liegen zwischen -1000 und +1000, mit einigen Ausreissern bei -2000 und +3000.

Leichte Schiefe: Das Modell unterschätzt teils höhere Preise.

Empfehlung: Ausreisser prüfen, weitere Features hinzufügen, ggf. Log-Transformation der Zielvariable.

### Error Analysis

Analyse the predictions. See which towns have the largest error.

In [ ]:
# get all predictions for the training set.
y_train_predict = random_forest_model.predict(X_train)
df_with_residual = pd.DataFrame(X_train, columns=X_train.columns, copy=True)
df_with_residual['recidual (error)'] = np.abs(y_train_predict - y_train)
df_with_residual['price'] = y_train
df_with_residual['predicted_price'] = y_train_predict
print(df_with_residual.head())
# Add text, postalcode and town name
# we use join instead of merge, because we 'join' on the index column and do not perform a merge using a specific column
df_with_residual = df_with_residual.join(df[['description_raw', 'bfs_name', 'postalcode', 'town']])

In [ ]:
# check which location has the largest errors.
df_with_residual[(df_with_residual['recidual (error)'] > 500)].groupby(['pop', 'bfs_name']).size().sort_values(ascending=False)

In [28]:

df_with_residual[(df_with_residual['recidual (error)'] > 500)]['description_raw'].to_csv('data_with_large_residuals.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)